In [1]:
#import lightgbm as lgb
import pandas as pd
import gc
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
gc.enable()
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv('train.csv')
print(train.shape)
YY = ['x','y','z','Vx','Vy','Vz']

train['error'] = np.abs(train['y_sim'] - train['y'])


low = train.error.mean()-train.error.std()*2
high = train.error.mean()+train.error.std()*2

train = train[train.error > low]
print(train.shape)
train = train[train.error < high]
print(train.shape)


Y = train[YY]
Y['x'] = Y.x-train.x_sim
Y['y'] = Y.y-train.y_sim
Y['z'] = Y.z-train.z_sim
Y['Vx'] = Y.Vx-train.Vx_sim
Y['Vy'] = Y.Vy-train.Vy_sim
Y['Vz'] = Y.Vz-train.Vz_sim
X = train.drop(YY+['id','epoch','error'],axis=1)
sub = pd.read_csv('submission.csv')

(649912, 15)
(649912, 16)
(625221, 16)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [3]:
Y

,x,y,z,Vx,Vy,Vz
0,-12.692409,-20.441543,13.262073,-0.000776,-0.003506,0.002050
1,-12.172318,-29.543301,21.276285,0.001114,-0.002801,0.003673
2,-6.825572,-34.527552,32.888826,0.002555,-0.000734,0.004387
3,1.368937,-33.237259,44.637815,0.003163,0.001440,0.003809
4,10.266521,-26.790192,53.492019,0.003125,0.002887,0.002466
5,18.532910,-17.556977,58.234299,0.002752,0.003483,0.000958
6,25.578862,-7.646524,58.980365,0.002260,0.003427,-0.000372
7,31.246395,1.493905,56.450035,0.001760,0.002960,-0.001396
8,35.579468,9.006475,51.529603,0.001301,0.002266,-0.002095
9,38.702491,14.451831,45.083306,0.000897,0.001474,-0.002495


In [4]:
'''
tr_epoch = pd.read_csv('train.csv',usecols=['epoch'])
te_epoch = pd.read_csv('test.csv',usecols=['epoch'])

tr_epoch.epoch = pd.to_datetime(tr_epoch.epoch,format='%Y-%m-%dT%H:%M:%S.%f')
te_epoch.epoch = pd.to_datetime(te_epoch.epoch,format='%Y-%m-%dT%H:%M:%S.%f')

X['month'] = tr_epoch.epoch.dt.month
X['day'] = tr_epoch.epoch.dt.day
X['hour'] = tr_epoch.epoch.dt.hour
X['minute'] = tr_epoch.epoch.dt.minute
X['second'] = tr_epoch.epoch.dt.second
X['dayofyear'] = tr_epoch.epoch.dt.dayofyear
X['dayofweek'] = tr_epoch.epoch.dt.dayofweek 
X['week'] = tr_epoch.epoch.dt.week


del tr_epoch
'''

"\ntr_epoch = pd.read_csv('train.csv',usecols=['epoch'])\nte_epoch = pd.read_csv('test.csv',usecols=['epoch'])\n\ntr_epoch.epoch = pd.to_datetime(tr_epoch.epoch,format='%Y-%m-%dT%H:%M:%S.%f')\nte_epoch.epoch = pd.to_datetime(te_epoch.epoch,format='%Y-%m-%dT%H:%M:%S.%f')\n\nX['month'] = tr_epoch.epoch.dt.month\nX['day'] = tr_epoch.epoch.dt.day\nX['hour'] = tr_epoch.epoch.dt.hour\nX['minute'] = tr_epoch.epoch.dt.minute\nX['second'] = tr_epoch.epoch.dt.second\nX['dayofyear'] = tr_epoch.epoch.dt.dayofyear\nX['dayofweek'] = tr_epoch.epoch.dt.dayofweek \nX['week'] = tr_epoch.epoch.dt.week\n\n\ndel tr_epoch\n"

In [5]:
TEST = pd.read_csv('test.csv')
TEST = TEST.drop(['id','epoch'],axis=1)

'''
TEST['month'] = te_epoch.epoch.dt.month
TEST['day'] = te_epoch.epoch.dt.day
TEST['hour'] = te_epoch.epoch.dt.hour
TEST['minute'] = te_epoch.epoch.dt.minute
TEST['second'] = te_epoch.epoch.dt.second

TEST['dayofyear'] = te_epoch.epoch.dt.dayofyear
TEST['dayofweek'] = te_epoch.epoch.dt.dayofweek 
TEST['week'] = te_epoch.epoch.dt.week

del te_epoch
'''

#i shuffled it because in histroy term it's not a time series. It's an accumulation in my opinion. 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=47,shuffle=True)
#validation if needed remove '_' aftern X_train_ and y_train_
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=47,shuffle=True)

In [6]:
TEST.head()

,sat_id,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,1,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774
1,1,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402
2,1,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412
3,1,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600
4,1,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724


In [7]:
X.head(2)

,sat_id,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,0,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
1,0,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468


In [8]:
train = lgb.Dataset(X_train,y_train)

NameError: name 'lgb' is not defined

In [81]:
test = lgb.Dataset(X_test,y_test)

In [106]:
params = {
    'boosting_type': 'gbdt',#'dart',
    'objective': 'regression',
    'metric': {'mape'},#{'l2','mape','mean_absolute_error'},#'mean_absolute_error'
    'num_leaves': 41,# lgb is growing leaf-wise let's make depth smaller#41 bigger
    'learning_rate': 0.0007,#very large learning rate
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'verbose': -1,
    'seed':47

    
}
gc.collect()

146

In [ ]:
for yy in YY:
    print('-------{}-------'.format(yy))
    train = lgb.Dataset(X_train,y_train[yy])
    test = lgb.Dataset(X_test,y_test[yy])
    #so many early rounds for mape cope with l2 (mse)
    gbm = lgb.train(params,train,num_boost_round=1000,valid_sets=test,early_stopping_rounds=20,verbose_eval=True)
    sub[yy] = gbm.predict(TEST)
    #validation
    print('\n----------VALIDATION SCORE (MAbsE) -----------\n')
    y_pred = gbm.predict(X_val)
    print(yy, " : ",mean_absolute_error(y_val[yy], y_pred),'\n')

-------x-------
[1]	valid_0's mape: 1.87016
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's mape: 1.86974
[3]	valid_0's mape: 1.86997
[4]	valid_0's mape: 1.86916
[5]	valid_0's mape: 1.86898
[6]	valid_0's mape: 1.86835
[7]	valid_0's mape: 1.86767
[8]	valid_0's mape: 1.86888
[9]	valid_0's mape: 1.86802
[10]	valid_0's mape: 1.86891
[11]	valid_0's mape: 1.87016
[12]	valid_0's mape: 1.87058
[13]	valid_0's mape: 1.87039
[14]	valid_0's mape: 1.87161
[15]	valid_0's mape: 1.87199
[16]	valid_0's mape: 1.87249
[17]	valid_0's mape: 1.87304
[18]	valid_0's mape: 1.87175
[19]	valid_0's mape: 1.87329
[20]	valid_0's mape: 1.87425
[21]	valid_0's mape: 1.87466
[22]	valid_0's mape: 1.87423
[23]	valid_0's mape: 1.87517
[24]	valid_0's mape: 1.87611
[25]	valid_0's mape: 1.87682
[26]	valid_0's mape: 1.87847
[27]	valid_0's mape: 1.87848
Early stopping, best iteration is:
[7]	valid_0's mape: 1.86767

----------VALIDATION SCORE (MAbsE) -----------

x  :  2636.3993152810494 

-------y-

[183]	valid_0's mape: 0.308077
[184]	valid_0's mape: 0.308052
[185]	valid_0's mape: 0.308021
[186]	valid_0's mape: 0.307995
[187]	valid_0's mape: 0.307969
[188]	valid_0's mape: 0.307941
[189]	valid_0's mape: 0.307916
[190]	valid_0's mape: 0.307873
[191]	valid_0's mape: 0.307847
[192]	valid_0's mape: 0.307816
[193]	valid_0's mape: 0.307789
[194]	valid_0's mape: 0.307749
[195]	valid_0's mape: 0.307724
[196]	valid_0's mape: 0.307683
[197]	valid_0's mape: 0.307657
[198]	valid_0's mape: 0.307618
[199]	valid_0's mape: 0.307604
[200]	valid_0's mape: 0.307578
[201]	valid_0's mape: 0.307555
[202]	valid_0's mape: 0.307533
[203]	valid_0's mape: 0.307507
[204]	valid_0's mape: 0.307482
[205]	valid_0's mape: 0.307444
[206]	valid_0's mape: 0.307422
[207]	valid_0's mape: 0.30739
[208]	valid_0's mape: 0.307355
[209]	valid_0's mape: 0.307312
[210]	valid_0's mape: 0.307288
[211]	valid_0's mape: 0.307257
[212]	valid_0's mape: 0.307222
[213]	valid_0's mape: 0.307198
[214]	valid_0's mape: 0.307171
[215]	val

[448]	valid_0's mape: 0.301505
[449]	valid_0's mape: 0.301481
[450]	valid_0's mape: 0.30146
[451]	valid_0's mape: 0.301441
[452]	valid_0's mape: 0.301418
[453]	valid_0's mape: 0.30139
[454]	valid_0's mape: 0.301375
[455]	valid_0's mape: 0.301348
[456]	valid_0's mape: 0.301329
[457]	valid_0's mape: 0.3013
[458]	valid_0's mape: 0.301276
[459]	valid_0's mape: 0.30125
[460]	valid_0's mape: 0.301235
[461]	valid_0's mape: 0.301209
[462]	valid_0's mape: 0.301187
[463]	valid_0's mape: 0.301162
[464]	valid_0's mape: 0.30114
[465]	valid_0's mape: 0.301121
[466]	valid_0's mape: 0.301092
[467]	valid_0's mape: 0.301079
[468]	valid_0's mape: 0.301065
[469]	valid_0's mape: 0.301035
[470]	valid_0's mape: 0.30102
[471]	valid_0's mape: 0.300992
[472]	valid_0's mape: 0.30097
[473]	valid_0's mape: 0.300956
[474]	valid_0's mape: 0.300935
[475]	valid_0's mape: 0.300913
[476]	valid_0's mape: 0.300893
[477]	valid_0's mape: 0.300874
[478]	valid_0's mape: 0.300852
[479]	valid_0's mape: 0.300838
[480]	valid_0's 

[715]	valid_0's mape: 0.296592
[716]	valid_0's mape: 0.296577
[717]	valid_0's mape: 0.296558
[718]	valid_0's mape: 0.29655
[719]	valid_0's mape: 0.296535
[720]	valid_0's mape: 0.296526
[721]	valid_0's mape: 0.296507
[722]	valid_0's mape: 0.296496
[723]	valid_0's mape: 0.296485
[724]	valid_0's mape: 0.296459
[725]	valid_0's mape: 0.296443
[726]	valid_0's mape: 0.296434
[727]	valid_0's mape: 0.296418
[728]	valid_0's mape: 0.296396
[729]	valid_0's mape: 0.296378
[730]	valid_0's mape: 0.296362
[731]	valid_0's mape: 0.296344
[732]	valid_0's mape: 0.296326
[733]	valid_0's mape: 0.296307
[734]	valid_0's mape: 0.2963
[735]	valid_0's mape: 0.296274
[736]	valid_0's mape: 0.296257
[737]	valid_0's mape: 0.296249
[738]	valid_0's mape: 0.296232
[739]	valid_0's mape: 0.296217
[740]	valid_0's mape: 0.296212
[741]	valid_0's mape: 0.296205
[742]	valid_0's mape: 0.296196
[743]	valid_0's mape: 0.296165
[744]	valid_0's mape: 0.296147
[745]	valid_0's mape: 0.296128
[746]	valid_0's mape: 0.296108
[747]	valid

[981]	valid_0's mape: 0.29288
[982]	valid_0's mape: 0.292869
[983]	valid_0's mape: 0.292856
[984]	valid_0's mape: 0.292843
[985]	valid_0's mape: 0.292828
[986]	valid_0's mape: 0.292819
[987]	valid_0's mape: 0.292804
[988]	valid_0's mape: 0.292793
[989]	valid_0's mape: 0.292777
[990]	valid_0's mape: 0.292767
[991]	valid_0's mape: 0.292763
[992]	valid_0's mape: 0.292753
[993]	valid_0's mape: 0.292742
[994]	valid_0's mape: 0.292728
[995]	valid_0's mape: 0.292709
[996]	valid_0's mape: 0.292695
[997]	valid_0's mape: 0.292682
[998]	valid_0's mape: 0.292667
[999]	valid_0's mape: 0.292649
[1000]	valid_0's mape: 0.292639
Did not meet early stopping. Best iteration is:
[1000]	valid_0's mape: 0.292639

----------VALIDATION SCORE (MAbsE) -----------

Vx  :  0.46507304335694705 

-------Vy-------
[1]	valid_0's mape: 0.318525
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's mape: 0.31849
[3]	valid_0's mape: 0.318455
[4]	valid_0's mape: 0.318419
[5]	valid_0's mape: 0.318383

...

[776]	valid_0's mape: 0.309841
[777]	valid_0's mape: 0.309831
[778]	valid_0's mape: 0.309832
[779]	valid_0's mape: 0.309831
[780]	valid_0's mape: 0.30983
[781]	valid_0's mape: 0.309828
[782]	valid_0's mape: 0.309828
[783]	valid_0's mape: 0.309819
[784]	valid_0's mape: 0.309808
[785]	valid_0's mape: 0.30981
[786]	valid_0's mape: 0.309799
[787]	valid_0's mape: 0.309797
[788]	valid_0's mape: 0.309798
[789]	valid_0's mape: 0.309794
[790]	valid_0's mape: 0.309792
[791]	valid_0's mape: 0.309786
[792]	valid_0's mape: 0.309788
[793]	valid_0's mape: 0.309788
[794]	valid_0's mape: 0.309776
[795]	valid_0's mape: 0.309766
[796]	valid_0's mape: 0.309768
[797]	valid_0's mape: 0.30976
[798]	valid_0's mape: 0.30976
[799]	valid_0's mape: 0.309752
[800]	valid_0's mape: 0.309745
[801]	valid_0's mape: 0.309736
[802]	valid_0's mape: 0.309737
[803]	valid_0's mape: 0.30973
[804]	valid_0's mape: 0.309725
[805]	valid_0's mape: 0.309722
[806]	valid_0's mape: 0.309719
[807]	valid_0's mape: 0.309709
[808]	valid_0

[37]	valid_0's mape: 0.252372
[38]	valid_0's mape: 0.252342
[39]	valid_0's mape: 0.252299
[40]	valid_0's mape: 0.252269
[41]	valid_0's mape: 0.252239
[42]	valid_0's mape: 0.252212
[43]	valid_0's mape: 0.252183
[44]	valid_0's mape: 0.25215
[45]	valid_0's mape: 0.252119
[46]	valid_0's mape: 0.252091
[47]	valid_0's mape: 0.252064
[48]	valid_0's mape: 0.252031
[49]	valid_0's mape: 0.252002
[50]	valid_0's mape: 0.251968
[51]	valid_0's mape: 0.251942
[52]	valid_0's mape: 0.251903
[53]	valid_0's mape: 0.251876
[54]	valid_0's mape: 0.251846
[55]	valid_0's mape: 0.251816
[56]	valid_0's mape: 0.251787
[57]	valid_0's mape: 0.25176
[58]	valid_0's mape: 0.251731
[59]	valid_0's mape: 0.251702
[60]	valid_0's mape: 0.25167
[61]	valid_0's mape: 0.251639
[62]	valid_0's mape: 0.251603
[63]	valid_0's mape: 0.251576
[64]	valid_0's mape: 0.25154
[65]	valid_0's mape: 0.251508
[66]	valid_0's mape: 0.25147
[67]	valid_0's mape: 0.251435
[68]	valid_0's mape: 0.251397
[69]	valid_0's mape: 0.251367
[70]	valid_0's 

[306]	valid_0's mape: 0.245281
[307]	valid_0's mape: 0.245259
[308]	valid_0's mape: 0.245235
[309]	valid_0's mape: 0.245214
[310]	valid_0's mape: 0.245195
[311]	valid_0's mape: 0.245174
[312]	valid_0's mape: 0.245154
[313]	valid_0's mape: 0.245135
[314]	valid_0's mape: 0.245113
[315]	valid_0's mape: 0.245092
[316]	valid_0's mape: 0.245064
[317]	valid_0's mape: 0.245044
[318]	valid_0's mape: 0.245024
[319]	valid_0's mape: 0.245006
[320]	valid_0's mape: 0.244987
[321]	valid_0's mape: 0.244967
[322]	valid_0's mape: 0.24494
[323]	valid_0's mape: 0.244918
[324]	valid_0's mape: 0.244892
[325]	valid_0's mape: 0.244871
[326]	valid_0's mape: 0.244849
[327]	valid_0's mape: 0.244831
[328]	valid_0's mape: 0.244811
[329]	valid_0's mape: 0.244793
[330]	valid_0's mape: 0.244774
[331]	valid_0's mape: 0.244744
[332]	valid_0's mape: 0.244719
[333]	valid_0's mape: 0.244698
[334]	valid_0's mape: 0.244677
[335]	valid_0's mape: 0.244661
[336]	valid_0's mape: 0.244642
[337]	valid_0's mape: 0.244621
[338]	val

[572]	valid_0's mape: 0.240291
[573]	valid_0's mape: 0.240272
[574]	valid_0's mape: 0.240243
[575]	valid_0's mape: 0.240228
[576]	valid_0's mape: 0.240205
[577]	valid_0's mape: 0.240191
[578]	valid_0's mape: 0.240175
[579]	valid_0's mape: 0.240159
[580]	valid_0's mape: 0.240145
[581]	valid_0's mape: 0.24013
[582]	valid_0's mape: 0.240115
[583]	valid_0's mape: 0.240104
[584]	valid_0's mape: 0.240093
[585]	valid_0's mape: 0.240074
[586]	valid_0's mape: 0.240064
[587]	valid_0's mape: 0.240049
[588]	valid_0's mape: 0.240036
[589]	valid_0's mape: 0.240023
[590]	valid_0's mape: 0.240005
[591]	valid_0's mape: 0.239994
[592]	valid_0's mape: 0.239984
[593]	valid_0's mape: 0.239974
[594]	valid_0's mape: 0.239957
[595]	valid_0's mape: 0.239946
[596]	valid_0's mape: 0.239929
[597]	valid_0's mape: 0.239911
[598]	valid_0's mape: 0.239887
[599]	valid_0's mape: 0.239871
[600]	valid_0's mape: 0.239857
[601]	valid_0's mape: 0.239838
[602]	valid_0's mape: 0.239823
[603]	valid_0's mape: 0.239813
[604]	val

[838]	valid_0's mape: 0.236096
[839]	valid_0's mape: 0.236082
[840]	valid_0's mape: 0.236058
[841]	valid_0's mape: 0.236051
[842]	valid_0's mape: 0.23603
[843]	valid_0's mape: 0.236015
[844]	valid_0's mape: 0.235994
[845]	valid_0's mape: 0.235982
[846]	valid_0's mape: 0.235963
[847]	valid_0's mape: 0.235945
[848]	valid_0's mape: 0.235932
[849]	valid_0's mape: 0.235926
[850]	valid_0's mape: 0.235911
[851]	valid_0's mape: 0.235899
[852]	valid_0's mape: 0.235883
[853]	valid_0's mape: 0.235875
[854]	valid_0's mape: 0.235861
[855]	valid_0's mape: 0.235849
[856]	valid_0's mape: 0.235829
[857]	valid_0's mape: 0.235807
[858]	valid_0's mape: 0.23579
[859]	valid_0's mape: 0.235785


In [16]:
sub.head()

,id,x,y,z,Vx,Vy,Vz
0,3927,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774
1,3928,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402
2,3929,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412
3,3930,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600
4,3931,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724


In [18]:
sub['x'] = sub.x+TEST.x_sim
sub['y'] = sub.y+TEST.y_sim
sub['z'] = sub.z+TEST.z_sim
sub['Vx'] = sub.Vx+TEST.Vx_sim
sub['Vy'] = sub.Vy+TEST.Vy_sim
sub['Vz'] = sub.Vz+TEST.Vz_sim

In [19]:
sub.head()

,id,x,y,z,Vx,Vy,Vz
0,3927,-40100.674042,-42710.260508,19160.323664,13.001446,-2.078292,2.432323
1,3928,-22111.302117,-43496.314560,21391.233976,15.232239,1.081826,0.940206
2,3929,-1716.205961,-39195.868495,21101.384627,16.557319,6.038489,-1.618237
3,3930,18626.835772,-27230.557274,16644.890701,14.547637,13.016865,-5.608800
4,3931,32304.600853,-6599.120120,6816.044586,5.821514,18.578660,-9.503173


In [36]:
sub.to_csv('submission1_12.csv',index=False)

Index(['id', 'epoch', 'sat_id', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim',
       'Vz_sim', 'x_sim_std', 'x_sim_dif_mean', 'y_sim_std', 'y_sim_dif_mean',
       'z_sim_std', 'z_sim_dif_mean', 'Vx_sim_std', 'Vx_sim_dif_mean',
       'Vy_sim_std', 'Vy_sim_dif_mean', 'Vz_sim_std', 'Vz_sim_dif_mean'],
      dtype='object')